In [1]:
import os
os.chdir('../')
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
from sklearn.ensemble import *
from sklearn.metrics import *
from sklearn.svm import *
from sklearn.linear_model import *
from sklearn.preprocessing import *
from sklearn.model_selection import *
from sklearn.ensemble import *
from sklearn.gaussian_process import *
from sklearn.naive_bayes import *
from sklearn.neighbors import *
from sklearn.neural_network import *
from sklearn.tree import *
import numpy as np
np.set_printoptions(suppress=True)
from sklearn.impute import *
import xgboost as xgb
import time
from sklearn.feature_selection import *

In [3]:
data = pd.read_csv('data/UNSW_NB15_training-set.csv',index_col='id')
data.head()

dur proto service state  spkts  dpkts  sbytes  dbytes  rate  sttl  dttl  \
id                                                                            
1  0.12   tcp       -   FIN      6      4     258     172 74.09   252   254   
2  0.65   tcp       -   FIN     14     38     734   42014 78.47    62   252   
3  1.62   tcp       -   FIN      8     16     364   13186 14.17    62   252   
4  1.68   tcp     ftp   FIN     12     12     628     770 13.68    62   252   
5  0.45   tcp       -   FIN     10      6     534     268 33.37   254   252   

       sload      dload  sloss  dloss  sinpkt  dinpkt      sjit      djit  \
id                                                                          
1  14,158.94   8,495.37      0      0   24.30    8.38     30.18     11.83   
2   8,395.11 503,571.31      2     17   49.91   15.43     61.43  1,387.78   
3   1,572.27  60,929.23      1      6  231.88  102.74 17,179.59 11,420.93   
4   2,740.18   3,358.62      1      3  152.88   90.24    259.08  4,991.78   
5   8,561.50   3,987.06      2      1   47.75   75.66  2,415.84    115.81   

    swin       stcpb       dtcpb  dwin  tcprtt  synack  ackdat  smean  dmean  \
id                                                                             
1    255   621772692  2202533631   255    0.00    0.00    0.00     43     43   
2    255  1417884146  3077387971   255    0.00    0.00    0.00     52   1106   
3    255  2116150707  2963114973   255    0.11    0.06    0.05     46    824   
4    255  1107119177  1047442890   255    0.00    0.00    0.00     52     64   
5    255  2436137549  1977154190   255    0.13    0.07    0.06     53     45   

    trans_depth  response_body_len  ct_srv_src  ct_state_ttl  ct_dst_ltm  \
id                                                                         
1             0                  0           1             0           1   
2             0                  0          43             1           1   
3             0                  0           7             1           2   
4             0                  0           1             1           2   
5             0                  0          43             1           2   

    ct_src_dport_ltm  ct_dst_sport_ltm  ct_dst_src_ltm  is_ftp_login  \
id                                                                     
1                  1                 1               1             0   
2                  1                 1               2             0   
3                  1                 1               3             0   
4                  1                 1               3             1   
5                  2                 1              40             0   

    ct_ftp_cmd  ct_flw_http_mthd  ct_src_ltm  ct_srv_dst  is_sm_ips_ports  \
id                                                                          
1            0                 0           1           1                0   
2            0                 0           1           6                0   
3            0                 0           2           6                0   
4            1                 0           2           1                0   
5            0                 0           2          39                0   

   attack_cat  label  
id                    
1      Normal      0  
2      Normal      0  
3      Normal      0  
4      Normal      0  
5      Normal      0

In [4]:
def preprocess_initial(df):
    df = df.drop(['attack_cat','proto','service','state'], axis=1)
    return df

In [5]:
df_final = preprocess_initial(data)
train_y  = df_final['label']
train_x  = df_final.drop(['label'],axis=1)

# General Correlation

In [6]:
train_x.corrwith(train_y)

dur                  0.04
spkts               -0.05
dpkts               -0.12
sbytes               0.02
dbytes              -0.08
rate                 0.34
sttl                 0.69
dttl                 0.10
sload                0.18
dload               -0.39
sloss               -0.00
dloss               -0.09
sinpkt              -0.18
dinpkt              -0.02
sjit                -0.01
djit                -0.06
swin                -0.33
stcpb               -0.26
dtcpb               -0.25
dwin                -0.32
tcprtt               0.08
synack               0.06
ackdat               0.10
smean               -0.01
dmean               -0.34
trans_depth          0.01
response_body_len   -0.02
ct_srv_src           0.23
ct_state_ttl         0.58
ct_dst_ltm           0.23
ct_src_dport_ltm     0.31
ct_dst_sport_ltm     0.36
ct_dst_src_ltm       0.30
is_ftp_login        -0.01
ct_ftp_cmd          -0.01
ct_flw_http_mthd     0.02
ct_src_ltm           0.24
ct_srv_dst           0.23
is_sm_ips_po

# Recursive Feature Elimination

In [7]:
estimator = RandomForestClassifier()
selector = RFE(estimator, n_features_to_select=1)
selector = selector.fit(train_x, train_y)
sorted([(i,j) for i,j in zip(selector.ranking_,train_x.columns)])

[(1, 'sttl'),
 (2, 'dload'),
 (3, 'ct_state_ttl'),
 (4, 'sbytes'),
 (5, 'rate'),
 (6, 'dur'),
 (7, 'sload'),
 (8, 'ct_srv_dst'),
 (9, 'dmean'),
 (10, 'smean'),
 (11, 'dttl'),
 (12, 'ackdat'),
 (13, 'tcprtt'),
 (14, 'ct_dst_src_ltm'),
 (15, 'dbytes'),
 (16, 'ct_srv_src'),
 (17, 'dinpkt'),
 (18, 'synack'),
 (19, 'sinpkt'),
 (20, 'sjit'),
 (21, 'djit'),
 (22, 'sloss'),
 (23, 'ct_src_ltm'),
 (24, 'dtcpb'),
 (25, 'dpkts'),
 (26, 'ct_dst_ltm'),
 (27, 'spkts'),
 (28, 'dloss'),
 (29, 'is_sm_ips_ports'),
 (30, 'stcpb'),
 (31, 'ct_dst_sport_ltm'),
 (32, 'ct_src_dport_ltm'),
 (33, 'swin'),
 (34, 'ct_flw_http_mthd'),
 (35, 'response_body_len'),
 (36, 'trans_depth'),
 (37, 'is_ftp_login'),
 (38, 'dwin'),
 (39, 'ct_ftp_cmd')]

# P value: Hypothesis

In [8]:
sorted([(i,j,k) for j,i,k in zip(f_classif(train_x, train_y)[0],f_classif(train_x, train_y)[1],train_x.columns)])

[(0.0, 1586.1773191397142, 'dloss'),
 (0.0, 1598.4953220666328, 'dttl'),
 (0.0, 1678.075240924856, 'ackdat'),
 (0.0, 2501.116244285493, 'dpkts'),
 (0.0, 5612.172675056767, 'sinpkt'),
 (0.0, 6066.435816129326, 'sload'),
 (0.0, 6191.368613438671, 'is_sm_ips_ports'),
 (0.0, 9618.746065756794, 'ct_srv_dst'),
 (0.0, 9707.760648034577, 'ct_srv_src'),
 (0.0, 9783.377995476476, 'ct_dst_ltm'),
 (0.0, 10549.386809010533, 'ct_src_ltm'),
 (0.0, 11723.168274085268, 'dtcpb'),
 (0.0, 12194.973687512549, 'stcpb'),
 (0.0, 17835.39975687167, 'ct_dst_src_ltm'),
 (0.0, 18059.21054636873, 'ct_src_dport_ltm'),
 (0.0, 19950.90966946118, 'dwin'),
 (0.0, 21961.79661995875, 'swin'),
 (0.0, 22611.817828177424, 'rate'),
 (0.0, 23195.03974946047, 'dmean'),
 (0.0, 25645.955323792747, 'ct_dst_sport_ltm'),
 (0.0, 32170.312850959333, 'dload'),
 (0.0, 87830.77949332567, 'ct_state_ttl'),
 (0.0, 161780.5269520456, 'sttl'),
 (1.2571046264447474e-256, 1174.8690712159632, 'tcprtt'),
 (5.495714663737611e-228, 1042.2596525813

# Feature Dependency

In [9]:
sorted([(i,j) for i,j in zip(mutual_info_classif(train_x, train_y),train_x.columns)],reverse=True)

[(0.46705365912127816, 'sbytes'),
 (0.38873324890468486, 'sttl'),
 (0.37244736671457623, 'dbytes'),
 (0.3600215700363121, 'ct_state_ttl'),
 (0.35574314056863976, 'dttl'),
 (0.35355767700008056, 'rate'),
 (0.347479983597782, 'sload'),
 (0.3389579636421891, 'dur'),
 (0.3338527128268829, 'smean'),
 (0.3135998237920705, 'dmean'),
 (0.29745216781056283, 'dinpkt'),
 (0.2811813997197048, 'dload'),
 (0.2738520531078188, 'dpkts'),
 (0.2481779197772649, 'sinpkt'),
 (0.23531474530310237, 'tcprtt'),
 (0.23447946242409334, 'synack'),
 (0.23435519578846464, 'ackdat'),
 (0.2076716155547027, 'sjit'),
 (0.1802155725333474, 'spkts'),
 (0.17862296986079462, 'djit'),
 (0.146027815188696, 'dloss'),
 (0.13847709505894357, 'ct_dst_sport_ltm'),
 (0.1193228111591742, 'sloss'),
 (0.11155787086827162, 'ct_dst_ltm'),
 (0.1000398423733031, 'ct_srv_dst'),
 (0.0940814538869037, 'ct_dst_src_ltm'),
 (0.09399518600071399, 'ct_src_ltm'),
 (0.089841568354329, 'ct_src_dport_ltm'),
 (0.08711984866705635, 'ct_srv_src'),
 (0

# Using Coeffecients

In [10]:
selector = SelectFromModel(LogisticRegressionCV(max_iter=500)).fit(train_x, train_y)
sorted([(np.abs(i),j) for i,j in zip(selector.estimator_.coef_[0],train_x.columns)],reverse=True)

[(1.7982092769594112e-05, 'rate'),
 (7.355304302596607e-06, 'dbytes'),
 (5.3125318221711235e-06, 'dload'),
 (3.94373427663374e-06, 'sbytes'),
 (1.3900534915577509e-06, 'response_body_len'),
 (8.148190414161315e-07, 'sinpkt'),
 (1.4628729952157284e-07, 'djit'),
 (1.1815902569983158e-07, 'sjit'),
 (5.4318026990835384e-08, 'sttl'),
 (2.5002306133874387e-08, 'dttl'),
 (1.750848200170318e-08, 'dmean'),
 (1.1467754390510569e-08, 'dinpkt'),
 (8.49268069732061e-09, 'smean'),
 (5.6541035183327795e-09, 'dwin'),
 (4.704717168445612e-09, 'dpkts'),
 (4.203740602134746e-09, 'swin'),
 (3.5676234540670647e-09, 'spkts'),
 (2.547060510924698e-09, 'dloss'),
 (2.4350362789653756e-09, 'ct_dst_src_ltm'),
 (2.212475926604141e-09, 'ct_srv_src'),
 (2.165729327491731e-09, 'ct_srv_dst'),
 (1.9488768503659644e-09, 'sload'),
 (1.9225829007041415e-09, 'ct_src_ltm'),
 (1.8662908657785194e-09, 'ct_src_dport_ltm'),
 (1.7285332460671222e-09, 'ct_dst_ltm'),
 (1.5936717834020963e-09, 'ct_dst_sport_ltm'),
 (1.193104735933